In [13]:
import numpy as np
import pandas as pd
import heapq as hp
from collections import Counter

df = pd.read_csv(u'../notes/data/iris.txt',sep=' ')

In [14]:
A = np.hstack([
        np.matrix(df.sl).T, 
        np.matrix(df.sw).T, 
        np.matrix(df.pl).T, 
        np.matrix(df.pw).T])

#categories
c = np.matrix(df.c).T

In [15]:
## Use a heap to store the smallest items

# Define an object and overload custom comparison operators
class tup:
    def __init__(self, val, idx):
        self.val = val
        self.idx = idx
        
    def __lt__(self, other):
        '''Redefine for max-heap'''
        return self.val > other.val
    
    def __le__(self, other):
        return self.val <= other.val
 
    def __eq__(self, other):
        return self.val == other.val
    
    def __ne__(self, other):
        return self.val != other.val

    def __gt__(self, other):
        return self.val > other.val

    def __ge__(self, other):
        return self.val >= other.val

    def __str__(self):
        return '{:.3},{:d}'.format(self.val,self.idx)

In [16]:
def findKNearestNeighbour(k,A,testData,c):
    heap = []
    N = A.shape[0]   
    
    # Heap with dummy data
    for i in range(k):
        hp.heappush(heap, tup(np.inf, -1))
    
    for i in range(N):
        e = A[i,:] - testData
        e = e.reshape((4,1))
        tp = tup(float(e.T*e), i)
        if tp <= heap[0]:
            hp.heapreplace(heap, tp)

    neighbourClasses = []
    for element in heap:
        neighbourClasses.append(int(c[element.idx]))
        
    resultingClass = Counter(neighbourClasses).most_common(1)[0][0]
    
    return heap, resultingClass

In [17]:
#Test data
x_test = A[1,:] + 5*np.random.randn(1,4)

x_test

matrix([[-4.75851006,  8.92731611, -3.6090935 ,  0.95598091]])

In [18]:
heap, resultingClass = findKNearestNeighbour(10,A,x_test,c)
resultingClass

1

In [19]:
# Display k nearest neighbours
print("Neighbours Data:")
for j in range(len(heap)):
    h = hp.heappop(heap)
    print(j,"Distance:",h.val,", Category: ",int(c[h.idx]))
print('\nResulting class is',resultingClass)

Neighbours Data:
0 Distance: 149.21378860128806 , Category:  1
1 Distance: 148.21366702603336 , Category:  1
2 Distance: 146.93626841615702 , Category:  1
3 Distance: 146.04638510450948 , Category:  1
4 Distance: 145.86937074786346 , Category:  1
5 Distance: 143.6820888263396 , Category:  1
6 Distance: 143.65426247788366 , Category:  1
7 Distance: 141.35116238246084 , Category:  1
8 Distance: 140.09794559868556 , Category:  1
9 Distance: 137.77725741841394 , Category:  1

Resulting class is 1


In [20]:
#find performance measures as the number of true positives, false positives, true negatives and false negatives
def measurePerformance(A, c, selectedClass):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(c)):
        heap,predictedClass = findKNearestNeighbour(10,A,A[i,:],c);
        if(int(c[i]) == predictedClass == selectedClass):
            TP += 1
        elif(predictedClass != int(c[i]) and int(c[i]) == selectedClass):
            FP += 1
        elif(predictedClass == int(c[i]) and int(c[i]) != selectedClass):
            TN += 1
        elif(predictedClass != int(c[i]) and int(c[i]) != selectedClass):
            FN += 1

    return(TP, FP, TN, FN)

In [21]:
TP, FP, TN, FN = measurePerformance(A, c, 2)
print('Acurracy:',(1.*(TP+TN)/(FP+TN+TP+FN)))
print('Precision:',(1.*TP/(TP+FP)))
print('Recall:',(1.*TP/(TP+FN)))

Acurracy: 0.98
Precision: 0.98
Recall: 0.9607843137254902
